## Monte Carlo

In [ ]:
a_file = open(r"D:\TCC\PYTHON\MELLO\Dados\resultados_samanthaTFs.pkl", "rb")
resultados_TFs = pickle.load(a_file)
a_file.close()

In [ ]:
def monte_carlo(df_fimo, df_exp, df_genoma, iteracoes):
    df_fimo_np = np.array(df_genoma.query("gene_id == {}".format(list(df_fimo["gene_id"]))).index)
    df_exp_np = np.array(df_genoma.query("gene_id == {}".format(list(df_exp["gene_id"]))).index)
    df_genoma_np = np.array(df_genoma.index)
    
    df_inter = np.intersect1d(df_fimo_np, df_exp_np)
    perc_ = df_inter.shape[0] / df_exp_np.shape[0]#(df1_data.shape[0] + df2_data.shape[0])

    len_exp = len(df_exp_np)
    perc_sample = []
    for i in range(iteracoes):
        sample = np.random.choice(df_genoma_np, len_exp, replace = False)
        
        df_new_inter = np.intersect1d(df_fimo_np, sample)
        
        perc_sample.append(df_new_inter.shape[0] / (sample.shape[0])) #(df1_new.shape[0] + df2_new.shape[0])) 
    
    print("Done!")
        
    return perc_, perc_sample

In [ ]:
n_boot = 20000
mc_TFs = {}
count = 0

for TF in ["list(resultados_TFs.keys())[:]"]: # dict_analise_TF.keys():
    a = time.time()
    mc_exps = {}

    try:
        print("-------------------------------------\n",TF)
        print(tfs_encontrados.query("ID == '{}'".format(TF))["Name"].values[0])
        TF_filter = resultados_TFs[TF]["filter"]
        print(len(TF_filter))
        mc_exps["sig_M"] = monte_carlo(TF_filter, sig_M_taegut1, TaeGut1, n_boot)
        mc_exps ["sig_F"] = monte_carlo(TF_filter, sig_F_taegut1, TaeGut1, n_boot)
        mc_exps["sig_20"] = monte_carlo(TF_filter, sig_20_taegut1, TaeGut1, n_boot)
        mc_exps["sig_50"] = monte_carlo(TF_filter, sig_50_taegut1, TaeGut1, n_boot)
        
        mc_TFs[TF] = mc_exps
    except:
        print("-------------------------------------\n",TF)
        print(tfs_encontrados.query("ID == '{}'".format(TF))["Name"].values[0])
        print("Falha")
        
    count += 1
    print(round(count/len(resultados_TFs.keys())*100), "%", "-", time.time()-a)
    

In [ ]:
len(mc_TFs.keys())

In [ ]:
a_file = open(r"D:\TCC\PYTHON\MELLO\Dados\montecarlo_sample_samanthaTFs_3.pkl", "wb")
pickle.dump(mc_TFs, a_file)
a_file.close()

## ZScore

In [ ]:
a_file = open(r"D:\TCC\PYTHON\MELLO\Dados\montecarlo_sample_samanthaTFs_3.pkl", "rb")
mc_TFs = pickle.load(a_file)
a_file.close()

In [ ]:
def zscore_mc(TF, sub_conj):
    perc_TF, mc_TF = mc_TFs[TF][sub_conj]
    
    Z_mc_TF = stats.zscore(mc_TF)
    Z_exp_TF = (perc_TF - np.mean(mc_TF)) / np.std(mc_TF)

    return Z_mc_TF, Z_exp_TF

zscore_mc_sigM_TFs, zscore_real_sigM_TFs = {}, {}
zscore_mc_sigF_TFs, zscore_real_sigF_TFs = {}, {}
zscore_mc_sig20_TFs, zscore_real_sig20_TFs = {}, {}
zscore_mc_sig50_TFs, zscore_real_sig50_TFs = {}, {}

for TF in mc_TFs.keys():
    zscore_mc_sigM_TFs[TF], zscore_real_sigM_TFs[TF] = zscore_mc(TF, "sig_M")
    zscore_mc_sigF_TFs[TF], zscore_real_sigF_TFs[TF] = zscore_mc(TF, "sig_F")
    zscore_mc_sig20_TFs[TF], zscore_real_sig20_TFs[TF] = zscore_mc(TF, "sig_20")
    zscore_mc_sig50_TFs[TF], zscore_real_sig50_TFs[TF] = zscore_mc(TF, "sig_50")

In [ ]:
def mk_montecarlo_df(zscore_mc_TFs, zscore_real_TFs, map_jasparID2geneName):
    monte_carlo_df = pd.DataFrame([zscore_mc_TFs.keys(), zscore_mc_TFs.values()]).T
    monte_carlo_df.columns = ["TF_id","zscore_mc"]
    monte_carlo_df["zscore_real"] = monte_carlo_df["TF_id"].map(zscore_real_TFs).astype(float)
#     monte_carlo_df["zscore_real"] = monte_carlo_df["zscore_real"].astype(float)
    monte_carlo_df["gene_name"] = monte_carlo_df["TF_id"].map(map_jasparID2geneName)
    monte_carlo_df["gene_id"] = monte_carlo_df["TF_id"].map(map_jasparID2geneID)
    
    return monte_carlo_df

Jaspar_PWMs = pd.read_csv(r"D:\TCC\PYTHON\MELLO\analise todos tfs\Dados\all_vertebrata_tfs_jaspar.csv")
map_jasparID2geneName = dict(zip(Jaspar_PWMs["ID"], Jaspar_PWMs["Name"]))
map_geneName2jasparID = dict(zip(Jaspar_PWMs["Name"].str.upper(), Jaspar_PWMs["ID"]))
universe_lists_annotation["PWMid"] = universe_lists_annotation["gene"].str.upper().map(map_geneName2jasparID)
map_jasparID2geneID = dict(zip(universe_lists_annotation["PWMid"], universe_lists_annotation["GeneID"]))


faltante_jasparID2geneName = {"MA1964.1":"SMAD2", "MA1965.1":"SP5", "MA1970.1":"TRPS1", "MA1992.1":"IKZF3", "MA1994.1":"NKX2-1"}
faltante_jasparID2geneID = {"MA1964.1":100225108, "MA1965.1":115495956, "MA1970.1":100226946, "MA1992.1":100217891, "MA1994.1":100227803}
map_jasparID2geneName.update(faltante_jasparID2geneName)
map_jasparID2geneID.update(faltante_jasparID2geneID)

montecarlo_sigM = mk_montecarlo_df(zscore_mc_sigM_TFs, zscore_real_sigM_TFs, map_jasparID2geneName)
montecarlo_sigF = mk_montecarlo_df(zscore_mc_sigF_TFs, zscore_real_sigF_TFs, map_jasparID2geneName)
montecarlo_sig20 = mk_montecarlo_df(zscore_mc_sig20_TFs, zscore_real_sig20_TFs, map_jasparID2geneName)
montecarlo_sig50 = mk_montecarlo_df(zscore_mc_sig50_TFs, zscore_real_sig50_TFs, map_jasparID2geneName)